<a href="https://colab.research.google.com/github/madhasuki/Capstone-Project-ML-Path/blob/main/Porn_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import All Packages
First thing we have to do is import all the packages that we will use later

Please take a look at the first code. If you don't have gdown package, you have to install it first. This package help you to download file from google drive. I use this package because i put the dataset in my google drive.


In [ ]:
#!pip install gdown
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gdown
import zipfile
import os
import shutil
import tensorflow as tf
from tensorflow import keras
import pathlib
from google.colab import files
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
from PIL import Image


## Download datasets
Download all the datasets from Google Drive. After download the data, we extract them using zipfile package and then we split them into training set and testing set.

In [ ]:
# download the datasets first
def download_data(url, output):
  gdown.download(url, output, quiet=False)
  # after we download the datasets, call the extract funtion to extract the dataset
  extract_dataset(output)

# this function is to extract dataset
def extract_dataset(filename):
  local_zip = filename
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall()
  
  zip_ref.close()
  os.remove(filename) # remove zip file after extract



# download not porn datasets and extract them
# download_data('https://drive.google.com/u/4/uc?id=12xtO-msdJ4adT9-ftp1zbmj5ZW99jups', 'not_porn_datasets.zip')

In [ ]:
# download datasets and extract them, after that we remove zip file
download_data('https://drive.google.com/u/0/uc?id=1OU5aZnp1oluBustLBllRt6WG4qCwGVEB', 'datasets.zip')

Downloading...
From: https://drive.google.com/u/0/uc?id=1OU5aZnp1oluBustLBllRt6WG4qCwGVEB
To: /content/datasets.zip
100%|██████████| 309M/309M [00:04<00:00, 70.9MB/s]


Because the not porn data split into several sub folders, we put them into the same folder

In [ ]:
# not_porn_folder ='./not_porn' 
# not_porn = os.listdir(not_porn_folder) # get all sub directory inside not porn directory
# for folder in not_porn:
#   sub_folder = os.path.join(not_porn_folder, folder)
#   sub_folder_path = os.listdir(sub_folder) # get all data/file from every sub directories 
#   for file in sub_folder_path:
#     old_path = os.path.join(sub_folder, file)
#     new_path = os.path.join(not_porn_folder, file)
#     shutil.move(old_path, new_path) # move all the data from sub directory into main directory
#   os.rmdir(sub_folder) # after we move the data into main directory, we remove the sub directory

Check the amount of data in the porn and not porn directory

In [ ]:
porn_dir = './porn'
not_porn_dir = './not_porn'

porn_datasets = os.listdir(porn_dir)
not_porn_datasets = os.listdir(not_porn_dir)

print('total porn images :', len(porn_datasets))
print('total not porn images :', len(not_porn_datasets))

total porn images : 10001
total not porn images : 10001


## Split Data
After we download the datasets, we split them into training and testing

First thing we need to do is create the directory for datasets. We split this directory into 2 sub directory, training and testng, after that we put porn and not porn dataset inside each directory 

In [ ]:
# we create the dataset directory first
base_dir = './datasets'

train_dir = os.path.join(base_dir, 'training')
testing_dir = os.path.join(base_dir, 'testing')

# directory with training porn and not porn picts
train_porn_dir = os.path.join(train_dir, 'porn')
train_not_porn_dir = os.path.join(train_dir, 'not_porn')

# directory with testing porn and not porn picts
test_porn_dir = os.path.join(testing_dir, 'porn')
test_not_porn_dir = os.path.join(testing_dir, 'not_porn')


isExist = os.path.exists(base_dir) #check if directory exist or not

if not isExist:
  os.makedirs(base_dir) # create datasets directory

  os.makedirs(train_dir) #create training directory
  os.makedirs(testing_dir) #create testing directory

  os.makedirs(train_porn_dir) #create training porn directory
  os.makedirs(train_not_porn_dir) #create training not porn directory

  os.makedirs(test_porn_dir) #create testing porn directory
  os.makedirs(test_not_porn_dir) #create testing not porn directory

After we create the directory, we move all the datasets from porn and not porn directory into datasets directory.
We split them into 5000 pict for training and testing for the rest

In [ ]:
def split_data(dataset, old_dir, train_dir, test_dir):
  train_data = dataset[:7500]
  test_data = dataset[7500:]

  move_data(train_data, old_dir, train_dir)
  move_data(test_data, old_dir, test_dir)


def move_data(data, old_dir, new_dir):
  for dt in data:
    old_data_path = os.path.join(old_dir, dt)
    new_data_path = os.path.join(new_dir, dt)

    isExist = os.path.exists(old_data_path)
    if isExist:
      shutil.move(old_data_path, new_data_path)

# move porn file into training set
split_data(porn_datasets, porn_dir, train_porn_dir, test_porn_dir)
os.rmdir(porn_dir) # remove old porn directory

# move not porn file into training set
split_data(not_porn_datasets, not_porn_dir, train_not_porn_dir, test_not_porn_dir)
os.rmdir(not_porn_dir) # remove old not porn directory

Check the amount of data inside dataset directory

In [ ]:
train_porn_sets = os.listdir(train_porn_dir)
test_porn_sets = os.listdir(test_porn_dir)

train_not_porn_sets = os.listdir(train_not_porn_dir)
test_not_porn_sets = os.listdir(test_not_porn_dir)

print('total train porn images :', len(train_porn_sets))
print('total test porn images :', len(test_porn_sets))

print('\ntotal train not porn images :', len(train_not_porn_sets))
print('total test not porn images :', len(test_not_porn_sets))

total train porn images : 7500
total test porn images : 2501

total train not porn images : 7500
total test not porn images : 2501


In [ ]:
class_names = ['porn', 'not_porn']
with open('labels.txt', 'w') as f:
    f.write('\n'.join(class_names))

# Train Model

In [ ]:
def create_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=1e-4),
                metrics=['accuracy'])
  
  return model

In [ ]:
# # Creating Callback Class

# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}):

#     # Check accuracy
#     if(logs.get('accuracy') > 0.94):

#       # Stop if threshold is met
#       print("\nStop Training !!!")
#       self.model.stop_training = True

# # Instantiate class
# callbacks = myCallback()

In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=100,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        testing_dir,
        target_size=(150, 150),
        batch_size=100,
        class_mode='binary')

Found 14998 images belonging to 2 classes.
Found 5002 images belonging to 2 classes.


In [ ]:
# Constant for epochs
EPOCHS = 100

# Create new model
# model = create_model()

In [ ]:
# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)
model.summary()
# Train the model with the new callback
# history = model.fit(
#       train_generator,
#       steps_per_epoch=150,
#       epochs=EPOCHS,
#       validation_data=validation_generator,
#       validation_steps=50,
#       verbose=2) 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [ ]:
model.save('./myModel.h5')
!zip -r ./PornRecognition.zip ./myModel/
files.download('./PornRecognition.zip')

## Export and Saved model to TFLite

In [ ]:
# Export the model
export_dir = 'saved_model/1'

tf.saved_model.save(model, export_dir)
optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.optimizations = [optimization]
tflite_model = converter.convert()

tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

files.download(tflite_model_file)
files.download('labels.txt')

!zip -r ./PornRecognition.zip ./saved_model
files.download('./PornRecognition2.zip')

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_acc(history):
  '''Plots the training and validation loss and accuracy from a history object'''
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training Loss')
  plt.plot(epochs, val_loss, 'b', label='Validation Loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [ ]:
# Plot the results of training with data augmentation
plot_loss_acc(history_with_aug)

## Check the model

This block code is to check if the created model has the expected result 

In [ ]:
import zipfile
import os
extract_dataset('./PornRecognition.zip')

In [ ]:
from tensorflow import keras
model = keras.models.load_model('./myModel')

In [ ]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='./' + fn
  img=image.load_img(path, target_size=(150, 150))
  
  x=image.img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0]>0.5:
    print(fn + " is a porn")
  else:
    print(fn + " is not a porn")
 

Saving prefix_GantMan_0005AA0D-22DE-4DF3-8447-108E30974315.jpg.jpeg to prefix_GantMan_0005AA0D-22DE-4DF3-8447-108E30974315.jpg.jpeg
Saving prefix_GantMan_0005E2D3-A7D7-4CDA-AB8D-20BA7FD5E093.jpg.jpeg to prefix_GantMan_0005E2D3-A7D7-4CDA-AB8D-20BA7FD5E093.jpg.jpeg
Saving prefix_GantMan_000692EA-5A12-4D17-9CC2-75B59DD7DB2B.jpg.jpeg to prefix_GantMan_000692EA-5A12-4D17-9CC2-75B59DD7DB2B.jpg.jpeg
Saving prefix_GantMan_0006B35A-6AC5-4EB0-87F4-16D2F2FE37CC.jpg.jpeg to prefix_GantMan_0006B35A-6AC5-4EB0-87F4-16D2F2FE37CC.jpg.jpeg
[0.6694779]
prefix_GantMan_0005AA0D-22DE-4DF3-8447-108E30974315.jpg.jpeg is a porn
[0.7666305]
prefix_GantMan_0005E2D3-A7D7-4CDA-AB8D-20BA7FD5E093.jpg.jpeg is a porn
[0.5902504]
prefix_GantMan_000692EA-5A12-4D17-9CC2-75B59DD7DB2B.jpg.jpeg is a porn
[0.9633449]
prefix_GantMan_0006B35A-6AC5-4EB0-87F4-16D2F2FE37CC.jpg.jpeg is a porn
